# Cлучайный лес
В этой части работы планируется вывести значимые для этого алгоритма признаки

## 0) Загрузка пакетов и данных

In [15]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import recall_score, precision_score

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
raw = pd.read_csv('curry1415.csv', header=0)

## 1) Feature engineering 

In [11]:
# Интересно, есть ли разница для алгоритма между переменными "координаты" и "расстояние" + "угол"?

# добавим более точную переменную расстояния
raw['dist'] = np.sqrt(raw['x']**2 + raw['y']**2)

# введем угол броска

loc_x_zero = raw['x'] == 0
raw['angle'] = np.array([0]*len(raw))
raw['angle'][~loc_x_zero] = np.arctan(raw['y'][~loc_x_zero] / raw['x'][~loc_x_zero])
raw['angle'][loc_x_zero] = np.pi / 2 
raw['angle']=np.absolute(raw['angle'])
raw2=raw.copy()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
# единая переменная оставшегося времени
raw['remaining_time'] = raw['minutes_remaining'] * 60 + raw['seconds_remaining']

# дропаем ненужные переменные
todrop = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', \
          'espn_game_id', 'minutes_remaining', 'seconds_remaining', 'shot_distance', 'x', 'y', 'defender_name']

for i in todrop:
    raw = raw.drop(i, 1)
    
# создаем дамми-переменные из категориальных
categorical_vars = ['action_type', 'shot_type', 'opponent', 'period']

for i in categorical_vars:
    raw = pd.concat([raw, pd.get_dummies(raw[i], prefix=i)], 1)
    raw = raw.drop(i, 1)
    
# делим выборку на объясняющие переменные и таргетируемую
train = raw.drop('shot_made_flag', 1)
train_y = raw['shot_made_flag']

In [13]:
# отсортируем по времени
sorted_raw = raw2.sort_values(['espn_game_id', 'period', 'minutes_remaining', 'seconds_remaining'], ascending=[1, 1, 0, 0])
sorted_raw.index=range(len(sorted_raw))

sorted_raw['previous1'] = np.zeros(len(sorted_raw)) 

for i,row in enumerate(sorted_raw[1:].iterrows()):
    if i>0:
        if sorted_raw.loc[i,'espn_game_id'] == sorted_raw.loc[i-1,'espn_game_id']:
            sorted_raw.loc[i,'previous1'] = sorted_raw.loc[i-1,'shot_made_flag']
            
sorted_raw['previous2'] = np.zeros(len(sorted_raw)) 

for i,row in enumerate(sorted_raw[1:].iterrows()):
    if i>1:
        if sorted_raw.loc[i,'espn_game_id'] == sorted_raw.loc[i-1,'espn_game_id'] == sorted_raw.loc[i-2,'espn_game_id']:
            if sorted_raw.loc[i-1,'shot_made_flag']==sorted_raw.loc[i-2,'shot_made_flag']==1:
                sorted_raw.loc[i,'previous2'] = raw.loc[i-1,'shot_made_flag']
                
sorted_raw['previous3'] = np.zeros(len(sorted_raw)) 

for i,row in enumerate(sorted_raw[1:].iterrows()):
    if i>2:
        if sorted_raw.loc[i,'espn_game_id'] == sorted_raw.loc[i-1,'espn_game_id'] == sorted_raw.loc[i-2,'espn_game_id']== sorted_raw.loc[i-3,'espn_game_id']:
            if sorted_raw.loc[i-1,'previous2']==sorted_raw.loc[i,'previous1']==1:
                sorted_raw.loc[i,'previous3'] = raw.loc[i-1,'shot_made_flag']
                
sorted_raw['previous1']=sorted_raw['previous1'].astype(int)
sorted_raw['previous2']=sorted_raw['previous2'].astype(int)
sorted_raw['previous3']=sorted_raw['previous3'].astype(int)

In [14]:
sorted_raw['remaining_time'] = sorted_raw['minutes_remaining'] * 60 + sorted_raw['seconds_remaining']
# дропаем ненужные переменные
todrop = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', \
          'espn_game_id', 'minutes_remaining', 'seconds_remaining', 'shot_distance', 'x', 'y', 'defender_name']

for i in todrop:
    sorted_raw = sorted_raw.drop(i, 1)

# создаем дамми-переменные из категориальных
categorical_vars = ['action_type', 'shot_type', 'opponent', 'period']
# categorical_vars = ['action_type', 'shot_type', 'opponent', 'period', 'previous1', 'previous2', 'previous3']
for i in categorical_vars:
    sorted_raw = pd.concat([sorted_raw, pd.get_dummies(sorted_raw[i], prefix=i)], 1)
    sorted_raw = sorted_raw.drop(i, 1)
# делим выборку на объясняющие переменные и таргетируемую
train2 = sorted_raw.drop('shot_made_flag', 1)
train2_y = sorted_raw['shot_made_flag']

## 2) Случайный лес

### 2.1) Без включения предыдущих бросков

In [46]:
# создаем объект
rf = RandomForestClassifier(n_estimators=900, max_depth=70)

# рандомно делим выборку
X_train, X_test, y_train, y_test = train_test_split(train, train_y, test_size=0.3, random_state=18)

# фитим
rf.fit(X_train, y_train)
pred1 = rf.predict(X_test)

In [47]:
pr=precision_score(y_test, pred1)
rc=recall_score(y_test, pred1)
acc=accuracy_score(y_test, pred1)
fmera=2*pr*rc/(pr+rc)
print(acc, rc, pr, fmera)

0.625468164794 0.553846153846 0.631578947368 0.590163934426


In [51]:
importance1=rf.feature_importances_
cols1 = train.columns.values
feature_dataframe1 = pd.DataFrame( {'features': cols1,
     'Random Forest feature importances': importance1,
    })

# отсортируем значимость по убыванию
sorted_imp1 = feature_dataframe1.sort_values(['Random Forest feature importances'], ascending=0)
sorted_imp1.index=range(len(feature_dataframe1))

sorted_imp1.head(10)

,Random Forest feature importances,features
0,0.119165,dist
1,0.100746,angle
2,0.099283,remaining_time
3,0.095377,defender_distance
4,0.091979,shot_clock
5,0.084971,touch_time
6,0.055551,dribbles
7,0.037262,action_type_Jump Shot
8,0.016067,period_3
9,0.014833,period_1


### 2.2) С включением предыдущих бросков

In [52]:
# создаем объект
rf = RandomForestClassifier(n_estimators=900, max_depth=70)

# рандомно делим выборку
X_train, X_test, y_train, y_test = train_test_split(train2, train2_y, test_size=0.3, random_state=18)

# фитим
rf.fit(X_train, y_train)
pred2 = rf.predict(X_test)

In [53]:
pr=precision_score(y_test, pred2)
rc=recall_score(y_test, pred2)
acc=accuracy_score(y_test, pred2)
fmera=2*pr*rc/(pr+rc)
print(acc, rc, pr, fmera)

0.653558052434 0.602409638554 0.635593220339 0.618556701031


In [38]:
importance2=rf.feature_importances_
cols2 = train2.columns.values
feature_dataframe2 = pd.DataFrame( {'features': cols2,
     'Random Forest feature importances': importance2,
    })

# отсортируем значимость по убыванию
sorted_imp2 = feature_dataframe2.sort_values(['Random Forest feature importances'], ascending=0)
sorted_imp2.index=range(len(feature_dataframe2))

#sorted_imp2.head(11)

## 3) Оптимизация параметров случайного леса
### 3.1) С помощью k-Fold кросс-валидации

In [39]:
# без предыдущих бросков
av_fm = []
range_n = [10, 50, 100, 150, 250, 500]
kf = KFold(n_splits=10)
for n in range_n:
    scores_n = []
    rf = RandomForestClassifier(n_estimators=n)
    for train_k, test_k in kf.split(train):
        rf.fit(train.iloc[train_k], train_y.iloc[train_k])
        pred = rf.predict(train.iloc[test_k])  
        pr=precision_score(train_y.iloc[test_k], pred)
        rc=recall_score(train_y.iloc[test_k], pred)
        fmera=2*pr*rc/(pr+rc)
        scores_n.append(fmera)
    av_fm.append(np.mean(scores_n))
print(av_fm)

[0.4311464150047602, 0.46162683722231745, 0.47043204413665007, 0.46893426471400462, 0.48022855227744471, 0.4838918105275497]


In [41]:
# с предыдущими бросками 
av_fm = []
range_n = [250, 500, 700, 900]
kf = KFold(n_splits=10)
for n in range_n:
    scores_n = []
    rf = RandomForestClassifier(n_estimators=n)
    for train_k, test_k in kf.split(train2):
        rf.fit(train2.iloc[train_k], train2_y.iloc[train_k])
        pred = rf.predict(train2.iloc[test_k])  
        pr=precision_score(train2_y.iloc[test_k], pred)
        rc=recall_score(train2_y.iloc[test_k], pred)
        fmera=2*pr*rc/(pr+rc)
        scores_n.append(fmera)
    av_fm.append(np.mean(scores_n))
print(av_fm)

[0.59851957357797203, 0.60048406425051526, 0.60889392331381453, 0.60396665935973182]


** 700 ** деревьев -- самое то

In [44]:
# с предыдущими бросками 
av_fm = []
range_m = [2,4,8,10,20,50, 70]
kf = KFold(n_splits=10)
for m in range_m:
    scores_m = []
    rf = RandomForestClassifier(n_estimators=700, max_depth=m)
    for train_k, test_k in kf.split(train2):
        rf.fit(train2.iloc[train_k], train2_y.iloc[train_k])
        pred = rf.predict(train2.iloc[test_k])  
        pr=precision_score(train2_y.iloc[test_k], pred)
        rc=recall_score(train2_y.iloc[test_k], pred)
        fmera=2*pr*rc/(pr+rc)
        scores_m.append(fmera)
    av_fm.append(np.mean(scores_m))
print(av_fm)

[0.46485224925039237, 0.52908906889627516, 0.5825767953236195, 0.5974461306443779, 0.60134996821003539, 0.6029118673161703, 0.61014036004979899]


** 50 ** -- самое то, для максимальной глубины

### 3.2) С помощью стратификации

In [42]:
# с предыдущими бросками 
av_fm = []
range_n = [50, 100, 250, 500, 700, 900]
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.30, random_state=88)
for n in range_n:
    scores_n = []
    rf = RandomForestClassifier(n_estimators=n)
    for train_k, test_k in sss.split(train2, train2_y):
        rf.fit(train2.iloc[train_k], train2_y.iloc[train_k])
        pred = rf.predict(train2.iloc[test_k])  
        pr=precision_score(train2_y.iloc[test_k], pred)
        rc=recall_score(train2_y.iloc[test_k], pred)
        fmera=2*pr*rc/(pr+rc)
        scores_n.append(fmera)
    av_fm.append(np.mean(scores_n))
print(av_fm)

[0.59193918336215068, 0.60072406257091482, 0.60274302123278978, 0.6021340363832679, 0.60731108718784454, 0.60813844611446277]


** 900 ** деревьев -- самое то

In [45]:
# с предыдущими бросками 
# глубина деревьев
av_fm = []
range_m = [2,4,8,10,20,50, 70]
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.30, random_state=88)
for m in range_m:
    scores_m = []
    rf = RandomForestClassifier(n_estimators=700, max_depth=m)
    for train_k, test_k in sss.split(train2, train2_y):
        rf.fit(train2.iloc[train_k], train2_y.iloc[train_k])
        pred = rf.predict(train2.iloc[test_k])  
        pr=precision_score(train2_y.iloc[test_k], pred)
        rc=recall_score(train2_y.iloc[test_k], pred)
        fmera=2*pr*rc/(pr+rc)
        scores_m.append(fmera)
    av_fm.append(np.mean(scores_m))
print(av_fm)

[0.49538072398341904, 0.55213517706095572, 0.59491710188266522, 0.59979558893504792, 0.60741186020291926, 0.60267394785216344, 0.60819621092883325]


** 70 ** -- самое то для глубины дерева

## 4) Валидация
### 4.1) Shuffle Split

In [54]:
# без предыдущих бросков
sp = ShuffleSplit(n_splits=10, test_size=0.30, random_state=88)
score=[]
rf = RandomForestClassifier(n_estimators=900, max_depth=70)
for train_index, test_index in sp.split(train):
    rf.fit(train.iloc[train_index], train_y.iloc[train_index])
    pred=rf.predict(train.iloc[test_index])
    pr=precision_score(train_y.iloc[test_index], pred)
    rc=recall_score(train_y.iloc[test_index], pred)
    fmera=2*pr*rc/(pr+rc)
    score.append(fmera)
print(np.mean(score))

0.598807864749


In [55]:
# с предыдущими бросками
sp = ShuffleSplit(n_splits=10, test_size=0.30, random_state=88)
score=[]
rf = RandomForestClassifier(n_estimators=900, max_depth=70)
for train_index, test_index in sp.split(train2):
    rf.fit(train2.iloc[train_index], train2_y.iloc[train_index])
    pred=rf.predict(train2.iloc[test_index])
    pr=precision_score(train2_y.iloc[test_index], pred)
    rc=recall_score(train2_y.iloc[test_index], pred)
    fmera=2*pr*rc/(pr+rc)
    score.append(fmera)
print(np.mean(score))

0.596379518008


### 4.2) Стратификация

In [57]:
# без предыдущих бросков
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.30, random_state=88)
score=[]
rf = RandomForestClassifier(n_estimators=900, max_depth=70)
for train_index, test_index in sss.split(train, train_y):
    rf.fit(train.iloc[train_index], train_y.iloc[train_index])
    pred=rf.predict(train.iloc[test_index])
    pr=precision_score(train_y.iloc[test_index], pred)
    rc=recall_score(train_y.iloc[test_index], pred)
    fmera=2*pr*rc/(pr+rc)
    score.append(fmera)
print(np.mean(score))

0.59241826272


In [59]:
# с предыдущими бросками
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.30, random_state=88)
score=[]
rf = RandomForestClassifier(n_estimators=900, max_depth=70)
for train_index, test_index in sss.split(train2, train2_y):
    rf.fit(train2.iloc[train_index], train2_y.iloc[train_index])
    pred=rf.predict(train2.iloc[test_index])
    pr=precision_score(train2_y.iloc[test_index], pred)
    rc=recall_score(train2_y.iloc[test_index], pred)
    fmera=2*pr*rc/(pr+rc)
    score.append(fmera)
print(np.mean(score))

0.608123915828


### 4.3) K-fold

In [60]:
# без предыдущих бросков
kf = KFold(n_splits=10, shuffle=True, random_state=88)
score=[]
rf = RandomForestClassifier(n_estimators=900, max_depth=70)
for train_index, test_index in kf.split(train):
    rf.fit(train.iloc[train_index], train_y.iloc[train_index])
    pred=rf.predict(train.iloc[test_index])
    pr=precision_score(train_y.iloc[test_index], pred)
    rc=recall_score(train_y.iloc[test_index], pred)
    fmera=2*pr*rc/(pr+rc)
    score.append(fmera)
print(np.mean(score))

0.612660736904


In [61]:
# с предыдущими бросками
kf = KFold(n_splits=10, shuffle=True, random_state=88)
score=[]
rf = RandomForestClassifier(n_estimators=900, max_depth=70)
for train_index, test_index in kf.split(train2):
    rf.fit(train2.iloc[train_index], train2_y.iloc[train_index])
    pred=rf.predict(train2.iloc[test_index])
    pr=precision_score(train2_y.iloc[test_index], pred)
    rc=recall_score(train2_y.iloc[test_index], pred)
    fmera=2*pr*rc/(pr+rc)
    score.append(fmera)
print(np.mean(score))

0.604203443811
